In [1]:
# import packages
import pandas as pd

# read in data
df = pd.read_csv(
    "/Users/robbiei2/Library/CloudStorage/Box-Box/Robbie's Shared Folder/Projects/Seizure Pattern/Data Tables/animal_eeg_main_record.csv"
)
df.head()

,eeg_id,cage_number,ear_tag,sex,year,start_date,end_date,computer,channel,injection_date,injection,cycle_length,cycle_group,diestrus_dates,estrus_dates,proestrus_dates,unlabeled_dates
0,221.0,1079.21,N,female,2019,1/30/19,2/17/19,1,6,12/6/18,KA_left,11.0,long,"2/11, 2/7, 2/2, 2/15","2/4, 2/14","2/14, 2/3","2/5, 2/6, 2/8"
1,225.0,1079.12,2L,female,2019,1/30/19,2/17/19,1,7,12/5/18,KA_left,9.0,long,"2/1, 2/2, 2/16, 2/13","2/6, 2/7","2/4, 2/14, 2/5",2/3
2,226.0,1080.21,N,female,2019,1/30/19,2/17/19,1,5,12/6/18,KA_right,16.0,long,"2/6, 2/7, 2/14, 2/1","2/5, 2/6",2/21,"2/2, 2/3, 2/4"
3,227.0,1080.11,L,female,2019,1/30/19,2/25/19,2,1,12/5/18,KA_right,10.0,long,"2/15, 2/10, 2/6, 2/2",2/4,"2/8, 2/11","2/5, 2/7, 2/9"
4,230.0,1080.12,2L,female,2019,1/30/19,2/17/19,1,8,12/6/18,KA_left,6.0,regular,"2/11, 2/7","2/6, 2/15, 2/1","2/14, 2/20","2/2, 2/3, 2/4, 2/5"


In [2]:
# create unique animal id
df["mouse_id"] = df["cage_number"].astype(str) + " " + df["ear_tag"]

# melt data
df_melt = df[
    [
        "mouse_id",
        "sex",
        "year",
        "computer",
        "channel",
        "injection",
        "cycle_group",
        "diestrus_dates",
        "estrus_dates",
        "proestrus_dates",
        "unlabeled_dates",
    ]
]
df_melt = pd.melt(
    df_melt,
    id_vars=[
        "mouse_id",
        "sex",
        "year",
        "computer",
        "channel",
        "injection",
        "cycle_group",
    ],
    value_vars=["diestrus_dates", "estrus_dates", "proestrus_dates", "unlabeled_dates"],
    var_name="cycle_stage",
    value_name="dates",
)

df_melt = df_melt.dropna(
    subset=["dates"]
)  # drop male rows with blank date columns for estrous cycle stage
# df_melt["cage_number"].unique()
df_melt.head()

,mouse_id,sex,year,computer,channel,injection,cycle_group,cycle_stage,dates
0,1079.21 N,female,2019,1,6,KA_left,long,diestrus_dates,"2/11, 2/7, 2/2, 2/15"
1,1079.12 2L,female,2019,1,7,KA_left,long,diestrus_dates,"2/1, 2/2, 2/16, 2/13"
2,1080.21 N,female,2019,1,5,KA_right,long,diestrus_dates,"2/6, 2/7, 2/14, 2/1"
3,1080.11 L,female,2019,2,1,KA_right,long,diestrus_dates,"2/15, 2/10, 2/6, 2/2"
4,1080.12 2L,female,2019,1,8,KA_left,regular,diestrus_dates,"2/11, 2/7"


In [3]:
# reformat data to get dates into their own rows in MMDDYYYY format
df_melt["dates_split"] = df_melt["dates"].str.split(",")  # split dates into list
df_melt = df_melt.explode("dates_split")  # make list of dates into its own column
df_melt["dates_split"] = df_melt["dates_split"].str.strip()  # remove whitespace
df_melt[["month", "day"]] = df_melt["dates_split"].str.split("/", expand=True)[[0, 1]]

# split the dates_split vector into two separate vectors for M and D
df_melt["month"] = df_melt["month"].str.zfill(2)  # pad M with a zero
df_melt["day"] = df_melt["day"].str.zfill(2)  # pad D with a zero
df_melt["date_MMDDYYYY"] = (
    df_melt["month"].astype(str)
    + df_melt["day"].astype(str)
    + df_melt["year"].astype(str)
)

df_cleaned = df_melt.drop_duplicates()  # remove duplicates
df_cleaned.head()

,mouse_id,sex,year,computer,channel,injection,cycle_group,cycle_stage,dates,dates_split,month,day,date_MMDDYYYY
0,1079.21 N,female,2019,1,6,KA_left,long,diestrus_dates,"2/11, 2/7, 2/2, 2/15",2/11,02,11,02112019
0,1079.21 N,female,2019,1,6,KA_left,long,diestrus_dates,"2/11, 2/7, 2/2, 2/15",2/7,02,07,02072019
0,1079.21 N,female,2019,1,6,KA_left,long,diestrus_dates,"2/11, 2/7, 2/2, 2/15",2/2,02,02,02022019
0,1079.21 N,female,2019,1,6,KA_left,long,diestrus_dates,"2/11, 2/7, 2/2, 2/15",2/15,02,15,02152019
1,1079.12 2L,female,2019,1,7,KA_left,long,diestrus_dates,"2/1, 2/2, 2/16, 2/13",2/1,02,01,02012019


In [4]:
# reorganize data
df_cleaned = df_cleaned.drop(columns=["dates", "dates_split"])
reordered_columns = [
    "mouse_id",
    "sex",
    "injection",
    "cycle_group",
    "cycle_stage",
    "computer",
    "channel",
    "month",
    "day",
    "year",
    "date_MMDDYYYY",
]
df_cleaned = df_cleaned.loc[:, reordered_columns]

# clarify estrous cycle stages
df_cleaned.loc[
    df_cleaned["cycle_stage"].str.contains("diestrus"), "cycle_stage"
] = "diestrus"
df_cleaned.loc[
    df_cleaned["cycle_stage"].str.contains("estrus"), "cycle_stage"
] = "estrus"
df_cleaned.loc[
    df_cleaned["cycle_stage"].str.contains("proestrus"), "cycle_stage"
] = "proestrus"
df_cleaned.loc[
    df_cleaned["cycle_stage"].str.contains("unlabeled"), "cycle_stage"
] = "unlabeled"

df_cleaned.head()

,mouse_id,sex,injection,cycle_group,cycle_stage,computer,channel,month,day,year,date_MMDDYYYY
0,1079.21 N,female,KA_left,long,estrus,1,6,02,11,2019,02112019
0,1079.21 N,female,KA_left,long,estrus,1,6,02,07,2019,02072019
0,1079.21 N,female,KA_left,long,estrus,1,6,02,02,2019,02022019
0,1079.21 N,female,KA_left,long,estrus,1,6,02,15,2019,02152019
1,1079.12 2L,female,KA_left,long,estrus,1,7,02,01,2019,02012019


In [5]:
# generate month as integer with no leading zeroes to match filepath
df_cleaned["month_int"] = df_cleaned["month"].astype(str)
df_cleaned["month_int"] = df_cleaned["month"].astype(int)

# generate a two-digit year for some inconsistently named files
df_cleaned["year_two_digit"] = df_cleaned["year"].astype(str).str[2:]
df_cleaned["date_MMDDYY"] = (
    df_cleaned["month"].astype(str)
    + df_cleaned["day"].astype(str)
    + df_cleaned["year_two_digit"]
)

# generate filepaths
df_cleaned["filepath_1"] = df_cleaned.apply(
    lambda row: f"/volumes/Expansion/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYYYY']}/Output/Analysis_Ch {row['channel']}.csv",
    axis=1,
)

df_cleaned["filepath_2"] = df_cleaned.apply(
    lambda row: f"/volumes/Expansion/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYYYY']}/Output/Analysis_ch {row['channel']}.csv",
    axis=1,
)

df_cleaned["filepath_3"] = df_cleaned.apply(
    lambda row: f"/Volumes/Expansion 1/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYYYY']}/Output/Analysis_Ch {row['channel']}.csv",
    axis=1,
)

df_cleaned["filepath_4"] = df_cleaned.apply(
    lambda row: f"/Volumes/Expansion/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYYYY']}/Output/Analysis_Ch {row['channel']}.csv",
    axis=1,
)

df_cleaned["filepath_5"] = df_cleaned.apply(
    lambda row: f"/Volumes/Expansion/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYY']}/Output/Analysis_Ch {row['channel']}.csv",
    axis=1,
)

df_cleaned["filepath_6"] = df_cleaned.apply(
    lambda row: f"/Volumes/Expansion 1/EEG recordings COMP {row['computer']}/{row['year']}/{row['month_int']}/{row['date_MMDDYY']}/Output/Analysis_ch {row['channel']}.csv",
    axis=1,
)

df_cleaned.head()

,mouse_id,sex,injection,cycle_group,cycle_stage,computer,channel,month,day,year,date_MMDDYYYY,month_int,year_two_digit,date_MMDDYY,filepath_1,filepath_2,filepath_3,filepath_4,filepath_5,filepath_6
0,1079.21 N,female,KA_left,long,estrus,1,6,02,11,2019,02112019,2,19,021119,/volumes/Expansion/EEG recordings COMP 1/2019/...,/volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...
0,1079.21 N,female,KA_left,long,estrus,1,6,02,07,2019,02072019,2,19,020719,/volumes/Expansion/EEG recordings COMP 1/2019/...,/volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...
0,1079.21 N,female,KA_left,long,estrus,1,6,02,02,2019,02022019,2,19,020219,/volumes/Expansion/EEG recordings COMP 1/2019/...,/volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...
0,1079.21 N,female,KA_left,long,estrus,1,6,02,15,2019,02152019,2,19,021519,/volumes/Expansion/EEG recordings COMP 1/2019/...,/volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...
1,1079.12 2L,female,KA_left,long,estrus,1,7,02,01,2019,02012019,2,19,020119,/volumes/Expansion/EEG recordings COMP 1/2019/...,/volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion/EEG recordings COMP 1/2019/...,/Volumes/Expansion 1/EEG recordings COMP 1/201...


In [6]:
import os

# Initialize df_access with the filepath columns from df_cleaned
df_access = df_cleaned.copy()

# Check access
df_access["filepath_1"] = df_access["filepath_1"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)
df_access["filepath_2"] = df_access["filepath_2"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)
df_access["filepath_3"] = df_access["filepath_3"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)
df_access["filepath_4"] = df_access["filepath_4"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)
df_access["filepath_5"] = df_access["filepath_5"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)
df_access["filepath_6"] = df_access["filepath_6"].apply(
    lambda x: os.path.isfile(x) if pd.notna(x) else False
)

# Check access for either 'filepath_upper' or 'filepath_lower'
df_access["access_any"] = (
    df_access["filepath_1"]
    | df_access["filepath_2"]
    | df_access["filepath_3"]
    | df_access["filepath_4"]
    | df_access["filepath_5"]
    | df_access["filepath_6"]
)

df_access.to_csv(
    "/Users/robbiei2/Library/CloudStorage/Box-Box/Robbie's Shared Folder/Projects/Seizure Pattern/Data Tables/df_access.csv"
)

In [7]:
# copy, rename, and paste files into new folder
import os
import shutil

# Initialize dataframe for files that cannot be found
df_not_found = pd.DataFrame(columns=df_cleaned.columns)

# Define the target folder
target_folder = "/Users/robbiei2/Library/CloudStorage/Box-Box/Robbie's Shared Folder/Projects/Seizure Pattern/EEG Recordings"  # replace with your target folder path

# Iterate over rows in the dataframe
for idx, row in df_cleaned.iterrows():
    file_found = False
    # Iterate over the filepaths for the current row
    for i in range(1, 7):
        filepath = row[f"filepath_{i}"]
        # Check if the file exists
        if os.path.isfile(filepath):
            file_found = True
            # Generate new filename
            new_filename = f"{row['mouse_id']}-{row['sex']}-{row['injection']}-{row['cycle_group']}-{row['cycle_stage']}-{row['computer']}-{row['channel']}-{row['date_MMDDYYYY']}{os.path.splitext(filepath)[1]}"
            new_filepath = os.path.join(target_folder, new_filename)
            # Copy and rename
            shutil.copyfile(filepath, new_filepath)
            break  # If file found, break the loop for filepaths
    if not file_found:  # If no file found for the current row
        df_not_found.loc[len(df_not_found)] = row  # Append the row to df_not_found


# If you want to save df_not_found to a csv
df_not_found.to_csv(
    "/Users/robbiei2/Library/CloudStorage/Box-Box/Robbie's Shared Folder/Projects/Seizure Pattern/Data Tables/df_not_found.csv",
    index=False,
)